In [2]:
%pip install openai praw


[notice] A new release of pip available: 22.3.1 -> 23.1.2
[notice] To update, run: pip3 install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [25]:
import openai

In [4]:
import praw


In [7]:
reddit = praw.Reddit(client_id=client_id,
                     client_secret=client_secret,
                     user_agent='sentiment analysis test')

In [ ]:
for submission in reddit.subreddit("stocks").hot(limit=5):
    print(submission.title)

In [9]:
subreddit_stocks = reddit.subreddit("stocks")

In [10]:
subreddit_stocks.title

'Stocks - Investing and trading for all'

In [11]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)

r/Stocks going dark poll ends with 67% in favor of a 3 day blackout
r/Stocks Daily Discussion & Fundamentals Friday Jun 09, 2023
Tech leaders are calling for an A.I. pause because they have no product ready, Palantir CEO says
WSJ - S&P 500 ends longest bear market since the 1940s and signals beginning of new bull market.
GM to use Tesla's charging network, joining Ford in leveraging EV leader's tech


In [12]:
for post in subreddit_stocks.hot(limit=5):
    print(post.id)

1453kuh
1450iow
1453tou
144ptye
144km3h


In [ ]:
for post in subreddit_stocks.hot(limit=5):
    print(post.title)
    submission = reddit.submission(post.id)
    #  print top 2 comments of each submission
    counter = 0
    for comment in submission.comments:
        print(comment.body)
        # if comment.body == '[deleted]' # check to see if comment is deleted
        counter += 1
        if counter == 2:
            break

### Now, create a function that will get the top N post titles and the top N comments for each post and format tham as prompt strings and add them to a dictionary.

In [17]:
def get_titles_and_comments(subreddit="stocks", limit=6,num_comments=3,skip_first=2):
    subreddit = reddit.subreddit(subreddit)
    title_and_comments = {}

    for counter,post in enumerate(subreddit.hot(limit=limit)):
        
        if counter < skip_first:
            continue
        
        counter += (1-skip_first)

        title_and_comments[counter] = ""
        # PRAW submission object
        submission = reddit.submission(post.id)
        title = post.title

        #  {0: 'Title: title\n\nComments: \n\n', 1: 'Title: title\n\nComments: \n\n'}
        title_and_comments[counter] += 'Title: '+title+'\n\n'
        title_and_comments[counter] += 'Comments: \n\n'

        comment_counter = 0
        for comment in submission.comments:
            if not comment.body == '[deleted]':
                title_and_comments[counter] += comment.body+'\n'
                comment_counter += 1
            
            if comment_counter == num_comments:
                break
      
    return title_and_comments

In [18]:
titles_and_comments = get_titles_and_comments()

In [19]:
titles_and_comments

{1: "Title: Tech leaders are calling for an A.I. pause because they have no product ready, Palantir CEO says\n\nComments: \n\nA tale as old as force has existed. When you're not good enough to get what you want on your own, you run to the government.\nHe isn't wrong.\nI ahould buy more pltr\n",
 2: 'Title: WSJ - S&P 500 ends longest bear market since the 1940s and signals beginning of new bull market.\n\nComments: \n\nLongest bear market since the 1940s? Then what was 2000-2002?\nGood to know. Everything fixed in a day.\nYeah, everything is just peachy. /s \n\nThis is all a ruse.\n',
 3: "Title: GM to use Tesla's charging network, joining Ford in leveraging EV leader's tech\n\nComments: \n\nChargepoint, electrify America, evgo, blink etc are all absolute trash compared to the ease, reliability and speed of the Tesla supercharger network.\nWhat does it mean GM/ford to start installing the charging port known as NACS instead of current CCS? Does that mean CCS charging will eventually be 

In [20]:
def create_prompt(title_and_comments):
    task="Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative, or neutral.\n\n"
    return task+title_and_comments

In [21]:
titles_and_comments[1]

"Title: Tech leaders are calling for an A.I. pause because they have no product ready, Palantir CEO says\n\nComments: \n\nA tale as old as force has existed. When you're not good enough to get what you want on your own, you run to the government.\nHe isn't wrong.\nI ahould buy more pltr\n"

In [22]:
print(create_prompt(titles_and_comments[1]))

Return the stock ticker or company name mentioned in the following title and comments and classify the sentiment around the company as positive, negative, or neutral.

Title: Tech leaders are calling for an A.I. pause because they have no product ready, Palantir CEO says

Comments: 

A tale as old as force has existed. When you're not good enough to get what you want on your own, you run to the government.
He isn't wrong.
I ahould buy more pltr



In [35]:
for key, title_with_comments in titles_and_comments.items():
    prompt = create_prompt(title_with_comments) 

    response = openai.Completion.create(engine='text-davinci-003',
                                        prompt=prompt,
                                        max_tokens=256,
                                        temperature=0,
                                        top_p=1.0
    )
    print(title_with_comments)
    print(f"Sentiment Report From OpenAI: {response['choices'][0]['text']}")
    print("--------------------------------------------------")

Title: Tech leaders are calling for an A.I. pause because they have no product ready, Palantir CEO says

Comments: 

A tale as old as force has existed. When you're not good enough to get what you want on your own, you run to the government.
He isn't wrong.
I ahould buy more pltr

Sentiment Report From OpenAI: 
Company Name: Palantir
Sentiment: Neutral
--------------------------------------------------
Title: WSJ - S&P 500 ends longest bear market since the 1940s and signals beginning of new bull market.

Comments: 

Longest bear market since the 1940s? Then what was 2000-2002?
Good to know. Everything fixed in a day.
Yeah, everything is just peachy. /s 

This is all a ruse.

Sentiment Report From OpenAI: 
Stock Ticker/Company Name: S&P 500
Sentiment: Neutral
--------------------------------------------------
Title: GM to use Tesla's charging network, joining Ford in leveraging EV leader's tech

Comments: 

Chargepoint, electrify America, evgo, blink etc are all absolute trash compared